In [262]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

In [215]:
df=pd.read_csv("glassdoor_jobs.csv")

In [216]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [217]:
df.columns
df.drop('Unnamed: 0',axis=1,inplace=True)

In [218]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [219]:
#removes rows which having salary as -1
df=df[df["Salary Estimate"]!="-1"]

In [220]:
#now work on salary estimate and split it into min,max and avg salary estimate

halfway=df["Salary Estimate"].apply(lambda x:re.sub('\(.+\)',"",x).strip().replace("K","").replace("$","").split("-"))
min=halfway.apply(lambda x:"".join([i for i in x[0] if i in string.digits]))
max=halfway.apply(lambda x:"".join([i for i in x[1] if i in string.digits]))


In [221]:
df["min_salary"]=min.astype(float)
df["max_salary"]=max.astype(float)
df["avg_salary"]=(df["min_salary"]+df["max_salary"])/2

In [243]:
# company name text only 

In [240]:
df["Company Nameb"]=df["Company Name"].apply(lambda x:re.search("[a-zA-Z1-9\. ]+",x).group())

In [244]:
# state field

In [246]:
df["state"]=df["Location"].apply(lambda x:x.split(",")[1])

In [247]:
# get the age of company

In [248]:
df["age"]=df["Founded"].apply(lambda x:x if x<0 else 2020-x)

In [259]:
#datapoint corresponding to oldest company
df.iloc[np.argmax(df["age"].values)]

Job Title                                               Data Scientist
Salary Estimate                            $95K-$161K (Glassdoor est.)
Job Description      Overview\n\n\nAt Sotheby’s we use data science...
Rating                                                             3.6
Company Name                                            Sotheby's\n3.6
Location                                                  New York, NY
Headquarters                                              New York, NY
Size                                            1001 to 5000 employees
Founded                                                           1744
Type of ownership                                     Company - Public
Industry                                          Auctions & Galleries
Sector                                                          Retail
Revenue                               $500 million to $1 billion (USD)
Competitors                                                         -1
min_sa

In [260]:
# parsing job description

In [319]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stopwords=stopwords.words('english')
from nltk.tokenize import sent_tokenize, word_tokenize

In [364]:
def preprocess(ss):
    ss=str(ss).lower().strip()
    ss=ss.lower().replace("\n","").replace("•","").replace("’","'")
    ss=re.sub("\W+"," ",ss)
    ss=re.sub("\d+"," ",ss)
    ss=[i for i in word_tokenize(ss) if len(i)>1]
    ss=[PorterStemmer().stem(i) for i in ss if i not in stopwords]
    ss=" ".join(ss)
    return ss

In [365]:
df["Job Descriptionb"]=df["Job Description"].apply(preprocess)

In [368]:
df.drop(['Founded','Salary Estimate','Job Description','Location'],axis=1,inplace=True)

In [370]:
df.drop(["Company Name"],axis=1,inplace=True)

In [380]:
df=df.rename({"Company Nameb":"Company Name","Job Descriptionb":"Job Description"},axis=1,inplace=True)

In [400]:
df=df[['Company Name','Job Title','Job Description','Rating', 'Headquarters', 'Size', 'Type of ownership',
       'Industry', 'Sector', 'Revenue', 'Competitors', 'min_salary',
       'max_salary', 'avg_salary', 'state', 'age']]

In [401]:
df.to_csv("cleaned_data.csv",index=False)